In [6]:
""" 
Reddit Crawler
https://praw.readthedocs.org/en/v3.1.0/
https://github.com/praw-dev/praw
https://github.com/reddit/reddit/wiki/API
In addition to reddits recommendations, your user_agent string should not contain the keyword bot.
"""

' \nReddit Crawler\nhttps://praw.readthedocs.org/en/v3.1.0/\nhttps://github.com/praw-dev/praw\nhttps://github.com/reddit/reddit/wiki/API\nIn addition to reddits recommendations, your user_agent string should not contain the keyword bot.\n'

In [23]:
# !pip install praw

In [24]:
"Getting Started"
import praw
r = praw.Reddit(user_agent='my_cool_application')
submissions = r.get_subreddit('opensource').get_hot(limit=5)
print [str(x) for x in submissions]

# user_name = "_Daimon_"
user_name = "babysnorlax"
user = r.get_redditor(user_name)

thing_limit = 10
gen = user.get_submitted(limit=thing_limit)
#gen = user.get_comments(limit=thing_limit)

karma_by_subreddit = {}
for thing in gen:
    subreddit = thing.subreddit.display_name
    karma_by_subreddit[subreddit] = (karma_by_subreddit.get(subreddit, 0) + thing.score)

import pprint
pprint.pprint(karma_by_subreddit)

['206 :: Last day to Save WiFi: Act Now To Save WiFi From The FCC - Keep OpenWr...', '8 :: How Debian Is Trying to Shut Down the CIA and Make Software Trustworthy ...', '10 :: Munich now a major contributor to open source', '3 :: Introduction to Matrix \xe2\x80\x94 Free & Open Source messenger with audio/video c...', '6 :: I made some software that helps you get paid to work on open source - fo...']
{}


In [59]:
# Use a reddit bot
import praw
r = praw.Reddit('Gilmore Girls related-question monitor by /u/username v 19.0. ')
r.login('username','password')

# Get subreddit
subreddit = r.get_subreddit('gilmoregirls')
for submission in subreddit.get_top(limit=2):
#     Test if it contains a Gilmore Girls-related question
    ggWords = ['lorelai','rory']
    op_text = submission.selftext.lower()
    has_gg = any(string in op_text for string in ggWords)
    
#     Look through comments from last top submission
#     print submission
    flat_comments = praw.helpers.flatten_tree(submission.comments)

# Create comments list for last top submission
comment_ls = [comment.body for comment in flat_comments]
# print comment_ls

14 :: 11 Actors Who Were on Gilmore Girls Before They Were Famous
11 :: marty...
[u"I hate how he pretends to not know Rory while dating Lucy. I don't understand it at all.", u"Marty is probably my favorite of all of Rory's guys, although they don't actually date.  Him pretending to not know Rory is stupid and immature, but at the same time, it's obvious that he still has feelings for Rory when he hits on her at the party.  So while he acted immaturely, I can see why he would be thrown off by seeing Rory again while he was dating Lucy, and just kind of ignore her out of self-preservation.\n\nAlso, while I don't approve of the way he acted, I still think his overall conduct is better than the other guys'.  He treats her much better than Jess does, and doesn't cheat on her or with her.", u"I agree! Marty had the best qualities of at least Dean, and Jess. He was sweet and thoughtful like Dean. But not too clingy. He didn't try to ruin or get in the middle of her relationship with Dean whe

In [92]:
# sentiment analysis around Marty, from Gilmore Girls
# http://www.laurentluce.com/posts/twitter-sentiment-analysis-using-python-and-nltk/
import nltk

pos_tweets = [('I agree! Marty had the best qualities of at least Dean, and Jess. He was sweet and thoughtful like Dean. But not too clingy. He didnt try to ruin or get in the middle of her relationship with Dean when they got back together like Jess would have. But he was smart like Jess, and could keep Rory on her toes without being an ass about it','positive')]
neg_tweets = [('I hate how he pretends to not know Rory while dating Lucy. I dont understand it at all','negative')]
tweets = []

def filter(tweets):
    tweets_filter=list()
    for words in tweets:
        words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
        tweets_filter.append(words_filtered)
    return tweets_filter

for (words, sentiment) in pos_tweets + neg_tweets:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
    tweets.append((words_filtered, sentiment))

tokenize_comments = filter(comment_ls)
# test_tweets = [(ls,'positive') for ls in tokenize_comments]
# print test_tweets

def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

word_features = get_word_features(get_words_in_tweets(tweets))

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

training_set = nltk.classify.apply_features(extract_features, tweets)
classifier = nltk.NaiveBayesClassifier.train(training_set)

print classifier.classify(extract_features(comment_ls[1].split()))

negative


In [93]:
comment_ls[1]

u"Marty is probably my favorite of all of Rory's guys, although they don't actually date.  Him pretending to not know Rory is stupid and immature, but at the same time, it's obvious that he still has feelings for Rory when he hits on her at the party.  So while he acted immaturely, I can see why he would be thrown off by seeing Rory again while he was dating Lucy, and just kind of ignore her out of self-preservation.\n\nAlso, while I don't approve of the way he acted, I still think his overall conduct is better than the other guys'.  He treats her much better than Jess does, and doesn't cheat on her or with her."

As you can tell, our training set is very limited, and there is not a clear sentiment interpretation from the test sentence as well.  However, it is a fun example of looking at a subreddit's comments and performing sentiment analysis using a Naive Bayes Classifier.